In [3]:
import sys
from os import path, mkdir, listdir, chdir
import pickle
import cv2 as cv
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
from skimage import io
import pandas as pd
import numpy as np

modulos_path = path.abspath('../minIA')
if modulos_path not in sys.path:
    sys.path.append(modulos_path)
    
from utiles import lectura_img


In [4]:
#Recibe lista de keypoints de una imagen, crea los objetos KP de openCV
def genKeyPoints( kp_img ):
    keypoints = list()
    for kp in kp_img:
        keypoints.append(cv.KeyPoint(kp[0], kp[1], kp[2]))
    return keypoints

## Visualización de descriptores
#### Rutas de ejecución

In [5]:
@interact_manual
def load_images(pickle_path='', dir_images=''):
    global image_names
    global image_dir
    global desc_imgs
    
    pickle_file = open(path.abspath(pickle_path), 'rb')  
    args = pickle.load(pickle_file)
    desc_imgs = pickle.load(pickle_file)
    pickle_file.close()
    
    image_names= [(desc_imgs[i]['name_img'], i, len(desc_imgs[i]['keypoints']))for i in range(len(desc_imgs))]
    image_dir= dir_images    
    desc_imgs=desc_imgs

    print('#Images: '+ str(len(image_names)))

interactive(children=(Text(value='', description='pickle_path'), Text(value='', description='dir_images'), But…

In [ ]:
@interact
def show_images(imagen=image_names[0:10]):
    print(imagen)
    imagen, i, kp_len=imagen
    imagen= path.abspath(image_dir + imagen)
    img = cv.imread(imagen)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    keypoints = genKeyPoints(desc_imgs[i]['keypoints'])
    img=cv.drawKeypoints(gray,keypoints,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(img)

## Visualización tópicos
#### Rutas de ejecución

In [3]:
@interact_manual
def load_topicos(topics_path='',centXimg='', descr_kp='', image_dir='', 
                img_dir_out=''):
    global topics
    global len_topics
    global images_descr
    global desc_kp
    global etiquetas
    global dir_out
    global dir_img
    
    topics = open(topics_path, 'r')
    topics= topics.readlines()
    len_topics=len(topics)
    
    pickle_file = open(centXimg,'rb')
    images_descr = pickle.load(pickle_file)
    pickle_file.close()
    
    pickle_file = open(path.abspath(descr_kp), 'rb')  
    args = pickle.load(pickle_file)
    desc_kp = pickle.load(pickle_file)
    pickle_file.close()
    
    dir_out=img_dir_out
    dir_img=image_dir

    print('#Topics: '+ str(len_topics))

interactive(children=(Text(value='', description='topics_path'), Text(value='', description='centXimg'), Text(…

In [37]:
def indicesTopicos (centroides, images_descr,threshold=0.7):
    tam_cents = len(centroides)
    centroides_= set(centroides) 
    img_index = 0
    images = list()
    for descrp in images_descr[0]:
        descp_ = dict([(ic,i) for i,ic in enumerate(descrp)])
        descp = set(descrp)
        tam_descp = len(descrp)
        inter=centroides_.intersection(descp)
        overlap=len(inter)/min(len(centroides),len(descp))
        if overlap > threshold:
            images.append((img_index,[ descp_[ic] for ic in inter]))
        #Overlaping
        #max_ = round( min(tam_descp, tam_cents)*.30 ) #Requiere un 30%
        #posc_index = 0
        #posc = list()
        #for cent in descrp:
        #    if cent in centroides:
        #        posc.append(posc_index)
        #    posc_index += 1
        #if len(posc) > max_ : #No puede ser >= porque max_ puede ser cero
        #    images.append((img_index ,posc)) #Añade una tupla, del indice de la imagen y los indices de los KP
        img_index += 1
    return images

In [57]:
@interact_manual
def choose_topicos(Topics=list(range(len_topics)), num_imgs= (0, int(len(images_descr)/1), 5), threshold = (0.0, 1.0, 0.01)):
    global lista_imgs
    global num_topic
    global name_images
    
    num_topic= Topics
    centroides = topics[Topics].split()[1:]
    centroides = [ int(cent.split(':')[0]) for cent in centroides]
    lista_imgs = indicesTopicos (centroides, images_descr[0:num_imgs], threshold=threshold)
    
    imgs_index=[imgs[0] for imgs in lista_imgs]
    aux=0
    name_images=[]
    for i in imgs_index:
        name_images.append((desc_kp[i]['name_img'],aux,i))
        aux+=1
        
    print('#Images: '+str(len(lista_imgs)))
    print('#Len topic: '+str(len(centroides)))

interactive(children=(Dropdown(description='Topics', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

In [55]:
@interact
def show_images_per_topic(img=name_images):
    img,aux,i=img
    imagen = path.abspath(dir_img + img)
    img = cv.imread(imagen)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    key_points= [desc_kp[i]['keypoints'][kp] for kp in lista_imgs[aux][1]]
    keypoints = genKeyPoints(key_points)
    img=cv.drawKeypoints(gray,keypoints,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(img)

interactive(children=(Dropdown(description='img', options=(('752811.jpg', 0, 6064), ('692764.jpg', 1, 9486), (…

In [ ]:
@interact
def show_images_per_topic2(img=listdir(dir_out)):
    chdir(dir_out)
    image=io.imread(path.abspath(img))
    plt.imshow(image)